In [13]:
import json, glob, sys

sys.path.append(r"D:\ig_pipeline")

from b1k_pipeline.utils import get_targets, PIPELINE_ROOT, parse_name

In [ ]:
lists = {}
selections = {}

for target in get_targets("combined"):
    lists[target] = json.loads((PIPELINE_ROOT / "cad" / target / "artifacts" / "object_list.json").read_text())["provided_objects"]
    path = PIPELINE_ROOT / "cad" / target / "artifacts" / "collision_selection.json"
    if path.exists():
        selections[target] = json.loads(path.read_text())
    else:
        selections[target] = {}

In [15]:
# Now move everything over
for tgt, objl in lists.items():
    for obj in objl:
        for tgt2, sels in selections.items():
            if tgt == tgt2:
                continue
                
            del_sel = []
            for obj2, sel in sels.items():
                if not parse_name(obj2).group("model_id") == obj.split("-")[-1]:
                    continue
                del_sel.append(obj2)
                selections[tgt][obj2] = sel
                
            for x in del_sel:
                del selections[tgt2][x]

In [ ]:
for target in get_targets("combined"):
    path = PIPELINE_ROOT / "cad" / target / "artifacts" / "collision_selection.json"
    path.write_text(json.dumps(selections[target], indent=4))